In [1]:
val githubActor  = System.getenv("GITHUB_ACTOR")
val githubToken = System.getenv("GITHUB_TOKEN")

## MNIST with CNN

### Setup Github Access
For accessing the package in Github, we need to set up the Github access token and actor. This is done by setting the `GITHUB_ACTOR` and `GITHUB_TOKEN` environment variables.

In [2]:
val githubActor  = System.getenv("GITHUB_ACTOR")
val githubToken = System.getenv("GITHUB_TOKEN")

### Setup Kotlin Jupyter Dependencies

In [6]:
USE {
    repositories {
        maven {
            url = "https://maven.pkg.github.com/sk-ai-net/skainet-jupyter"
            credentials {
                username = githubActor
                password = githubToken
            }
        }
        mavenCentral()
    }
    dependencies {
        implementation("sk.ai.net:kotlin-jupyter:0.0.7")
        implementation("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.10.2")
        val ktorVersion = "3.1.3"
        implementation("io.ktor:ktor-client-cio:$ktorVersion")
        implementation("io.ktor:ktor-client-core:$ktorVersion")
        implementation("io.ktor:ktor-client-content-negotiation:$ktorVersion")

    }
}

### Importing Libraries

In [7]:
// SKaiNET
import sk.ai.net.Tensor
import sk.ai.net.dsl.network
import sk.ai.net.nn.activations.ReLU
import sk.ai.net.nn.Module
import sk.ai.net.Shape
import sk.ai.net.impl.DoublesTensor


// SKaiNET - MNIST
import sk.ai.net.io.data.mnist.MNISTLoaderFactory
import sk.ai.net.io.data.mnist.MNISTConstants
import sk.ai.net.io.data.mnist.MNISTImage
import sk.ai.net.io.data.mnist.MNISTDataset


// Kotlin
import kotlin.random.Random
import kotlinx.coroutines.runBlocking

### Download MNIST dataset

In [9]:
val (trainingData, testData) = runBlocking {
    MNISTLoaderFactory.create().let { loader ->
        loader.loadTrainingData() to loader.loadTestData()
    }
}

Using cached file: mnist-data/train-images-idx3-ubyte
Using cached file: mnist-data/train-labels-idx1-ubyte
Using cached file: mnist-data/t10k-images-idx3-ubyte
Using cached file: mnist-data/t10k-labels-idx1-ubyte
